# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
spaceship.shape

(8693, 14)

In [4]:
spaceship.dtypes

,0
PassengerId,object
HomePlanet,object
CryoSleep,object
Cabin,object
Destination,object
Age,float64
VIP,object
RoomService,float64
FoodCourt,float64
ShoppingMall,float64


In [5]:
spaceship = spaceship.dropna(how='any')

In [6]:
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])
spaceship = spaceship.drop(columns=['PassengerId', 'Name'])
spaceship_dummies = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination'])
spaceship_dummies.head()

<ipython-input-6-3940e1d50c0d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x[0])


,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,39.0,False,0.0,0.0,0.0,0.0,0.0,False,False,True,...,True,False,False,False,False,False,False,False,False,True
1,24.0,False,109.0,9.0,25.0,549.0,44.0,True,True,False,...,False,False,False,False,True,False,False,False,False,True
2,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,False,True,...,False,False,False,False,False,False,False,False,False,True
3,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,False,True,...,False,False,False,False,False,False,False,False,False,True
4,16.0,False,303.0,70.0,151.0,565.0,2.0,True,True,False,...,False,False,False,False,True,False,False,False,False,True


In [7]:
features = spaceship_dummies.drop(columns='Transported')
target = spaceship_dummies['Transported']
X_train, X_test, Y_train, Y_test = train_test_split(features, target)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

gradient = GradientBoostingClassifier()
gradient.fit(X_train, Y_train)

GradientBoostingClassifier()

- Evaluate your model

In [9]:
gradient.score(X_test, Y_test)

0.7953995157384988

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [10]:
from sklearn.model_selection import GridSearchCV

parameters = {'learning_rate':[0.05, 0.1], 'n_estimators':[50, 100, 150]}

- Run Grid Search

In [13]:
clf = GridSearchCV(gradient, parameters)
clf.fit(X_train, Y_train)

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.05, 0.1],
                         'n_estimators': [50, 100, 150]})

- Evaluate your model

In [16]:
clf.best_params_

{'learning_rate': 0.1, 'n_estimators': 50}

In [17]:
tuned_gradient = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50)
tuned_gradient.fit(X_train, Y_train)
tuned_gradient.score(X_test, Y_test)

0.7887409200968523